# Proyecto 5: Análisis de sentimiento en reviews de películas

En este proyecto vamos a trabajar con un dataset de reviews de películas. El objectivo es clasificar cada comentario de acuerdo al sentimiento positivo o negativo hacia la película. Para hacerlo, vamos a trabajar con procesamiento de texto para luego aplicar las diferentes técnicas predictivas que hemos visto.

## Dataset

En primer lugar, carguemos las reviews para comenzar a procesarlas. En este caso no tenemos un archivo `.csv`, sino un directorio estructurado de la siguiente forma: 
```
movie_reviews/
  neg/
  pos/
```
`sklearn` nos provee de la función `load_files` que permite recorrer este tipo de estructuras. 

In [23]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
import pandas as pd
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

Ahora en la variable `movies` tendremos guardadas las reviews (`movies.data`) y su sentimiento (`movies.target`).
Veamos cuantas reviews tenemos en total:

In [24]:
print("Tenemos {} reviews, con clases {},".format(len(movie_reviews.data), movie_reviews.target_names))

Tenemos 2000 reviews, con clases ['neg', 'pos'],


Analizando la estructura de los documentos, veamos como luce una muestra de `movies.data`:

In [25]:
movie_reviews.data[0][:500]

b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so cal"

Y ahora, con la propiedad `target` podemos ver la categoría asociada a esta review

In [26]:
movie_reviews.target[0]

0

En este caso es un 0, es decir `neg`.

## Separando en conjuntos de entrenamiento y de testeo

Antes de comenzar con el entrenamiento de los modelos probando distintos algoritmos para encontrar los mejores modelos, vamos a separar el conjunto de reviews en training y testing. 

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

## Preprocesamiento

In [28]:
print(type(X_train))

<class 'list'>


Para trabajar con texto tenemos que realizar una serie de pasos antes de poder entrenar un modelo. Es decir, buscar la forma de convertir el texto a una representación numérica que pueda ser interpretable por los algoritmos de clasificación. 
Para ello vamos a realizar una serie de pasos.
- Tokenizar el texto
- Convertir a vectores de términos/documentos
- Aplicar tfidf


Es importante destacar que el `fit` debe hacerse sobre el conjunto de `train` y no sobre el total, ya que `tfidf` tiene en cuenta la frecuencia de aparición de las palabras respecto al total del conjunto. Una vez que usamos el `fit` con el conjunto de entrenamiento podemos aplicar la transformación al conjunto de `test`. 

**1) Vectorizá** los textos para obtener una matriz de términos/documentos utilizando td-idf. 

In [29]:
c = X_train[0]
text = [c.decode('utf-8') for c in X_train]
len(text)
print(type(text[4]))


<class 'str'>


In [30]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stops=set(stopwords.words('english'))

#tokenizer = RegexpTokenizer(r'\w+')
#text=[tokenizer.tokenize(t) for t in text]
#print(text[0])
#corpus_without_stops = [[word.lower() for word in txt if word not in stops] for txt in text]
#text = corpus_without_stops

#Stemmer
#from nltk.stem import SnowballStemmer
#weno hay que castear el text para poder ir instancia por instancia de las 1600 reviews
#stemmer = SnowballStemmer("english")
#stemmed_text = [[stemmer.stem(i) for i in txt] for txt in text] #[stemmer.stem(i) for i in text]
#text = stemmed_text

#Frecuencia de palabras

#import nltk
#freq = [[nltk.FreqDist() for word in txt] for txt in text]
#castear, same arriba blablabla
#sorted_by_value = sorted(freq.items(), key=lambda kv: kv[1], reverse=True)
#for key,val in sorted_by_value:
#    print (str(key) + ':' + str(val))

#no correrlo

In [31]:
#from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = CountVectorizer()

#print(vectorizer.fit_transform(freq).todense()[0])
#print(vectorizer.vocabulary_)

#no correrlo, el tfid lo hace

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words=stops)
tfidf_matrix = vectorizer.fit_transform(text) 
print(tfidf_matrix.todense()[0])
print(vectorizer.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]]
{'arguably': 2031, 'anticipated': 1783, 'movie': 21123, 'century': 5364, 'george': 13333, 'lucas': 19175, 'achieved': 815, 'mixed': 20730, 'reaction': 25871, 'sure': 31426, 'blockbuster': 3716, 'despite': 8674, 'fact': 11560, 'script': 28230, 'plot': 24153, 'bit': 3551, 'let': 18573, 'entire': 10796, 'whole': 35434, 'considered': 6864, 'milestone': 20460, 'technical': 32005, 'excellence': 11183, 'sound': 29994, 'special': 30113, 'effects': 10236, 'department': 8532, 'terms': 32174, 'characters': 5482, 'far': 11697, 'outstanding': 22708, 'interesting': 16722, 'queen': 25491, 'amidala': 1538, 'natalie': 21455, 'portman': 24412, 'whose': 35452, 'mature': 19914, 'performance': 23558, 'exceeds': 11180, 'tender': 32120, '14': 54, 'years': 35970, 'age': 1151, 'one': 22370, 'hot': 15411, 'babe': 2619, 'defines': 8321, 'true': 33246, 'woman': 35681, 'leadership': 18386, 'presence': 24734, 'knows': 17949, 'take': 31788, 'command': 6416, 'situation': 29279, 'save': 27900

In [33]:
print(tfidf_matrix)

  (0, 2031)	0.07302260615066106
  (0, 1783)	0.07579403633478239
  (0, 21123)	0.03434735342868037
  (0, 5364)	0.056084365547199415
  (0, 13333)	0.10060325106497031
  (0, 19175)	0.13747821423440837
  (0, 815)	0.06920731590766134
  (0, 20730)	0.06622109388885983
  (0, 25871)	0.06224796379536384
  (0, 31426)	0.0357061240483896
  (0, 3716)	0.06224796379536384
  (0, 8674)	0.040728683086562606
  (0, 11560)	0.09177586423470653
  (0, 28230)	0.031838662692210355
  (0, 24153)	0.024982453976369123
  (0, 3551)	0.0701770941899973
  (0, 18573)	0.03794074263556953
  (0, 10796)	0.038547910149276145
  (0, 35434)	0.03566408189539526
  (0, 6864)	0.05519303457778563
  (0, 20460)	0.0909667911331315
  (0, 32005)	0.05723593532315542
  (0, 11183)	0.09348480436147584
  (0, 29994)	0.046058568216340426
  (0, 30113)	0.03755030429202241
  :	:
  (1599, 34070)	0.05682685227836108
  (1599, 28180)	0.05999196618914811
  (1599, 29552)	0.05999196618914811
  (1599, 8486)	0.05999196618914811
  (1599, 5250)	0.059991966189148

### Dimensión de la matriz de términos
Una vez vectorizados los documentos veamos qué forma tiene la matriz resultante

In [34]:
#X_train.shape veo la shape de mi X_train aka tfidf_matrix
print(tfidf_matrix.shape)
X_train = tfidf_matrix


(1600, 36199)


__¿ Por qué tiene estas dimensiones ?__

Responder.

Porque muestra el mapeo de términos en las distintas reviews en relación con su ubicación en el texto.

Porque el resultado es una matriz de términos por documentos. Por cada término existente en el dataset habrá una columna, ya que no se realizó ningún tipo de filtrado.

## Entrenando los modelos
Una vez que tenemos una matriz con los vectores correspondientes a los documentos luego de aplicarse td-idf podemos comenzar con el proceso de entrenamiento de los modelos. Para eso, vamos a comparar diferentes algoritmos vistos como SVM, Random Forest, AdaBoost, Voting. Y todos los algoritmos complementarios que se quieran introducir. 
Para el entrenamiento de los modelos usaremos GridSearch recorriendo una lista de parámetros según corresponda a cada algoritmo. Además usaremos 5-fold CV y el scoring estará dado por el área bajo la curva roc.

### Support Vector Machines (SVM)

__Ajustar un SVM (kernel lineal) al conjunto de datos de entrenamiento y devolver la matriz de confusión en testing.__

Basándonos en el paper https://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf, vamos a utilizar un kernel lineal. Al tener menos instancias que features este es el procedimiento recomendado. 

Nuestro espacio de parámetros para recorrer de forma exhaustiva utilizando GridSearch será el siguiente:

* C = [0.001, 0.01, 0.1, 1, 10]
* loss = ['hinge', 'squared_hinge'] 

**2) Entrenar** un modelo utilizando `Grid Search` y el clasificador `LinearSVC` con `5-fold` cross validation y que utilice como `scoring` el área bajo la curva roc.

In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

param_grid = [
    {'C':[0.001, 0.01, 0.1, 1, 10], 'loss' : ['hinge', 'squared_hinge']}
]

svc = LinearSVC()

grid_search = GridSearchCV(svc, param_grid, cv=5,
                           scoring='roc_auc', 
                           return_train_score=True)
grid_search.fit(tfidf_matrix, y_train)


GridSearchCV(cv=5, estimator=LinearSVC(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10],
                          'loss': ['hinge', 'squared_hinge']}],
             return_train_score=True, scoring='roc_auc')

In [36]:
grid_search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__class_weight', 'estimator__dual', 'estimator__fit_intercept', 'estimator__intercept_scaling', 'estimator__loss', 'estimator__max_iter', 'estimator__multi_class', 'estimator__penalty', 'estimator__random_state', 'estimator__tol', 'estimator__verbose', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

Veamos los resultados promediados por cada combinación de parámetros:

In [37]:
#sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score) no está disponible para esta versión de scikit learn

results = pd.DataFrame(grid_search.cv_results_, columns=['rank_test_score','params','mean_test_score','std_test_score','mean_train_score','std_train_score'])
results.sort_values(by=['rank_test_score'],inplace=True)
results.head()

rank_test_score                               params  mean_test_score  \
7                1    {'C': 1, 'loss': 'squared_hinge'}         0.921859   
9                2   {'C': 10, 'loss': 'squared_hinge'}         0.920891   
6                3            {'C': 1, 'loss': 'hinge'}         0.920656   
8                4           {'C': 10, 'loss': 'hinge'}         0.920258   
5                5  {'C': 0.1, 'loss': 'squared_hinge'}         0.905953   

   std_test_score  mean_train_score  std_train_score  
7        0.009618          1.000000         0.000000  
9        0.009274          1.000000         0.000000  
6        0.009191          0.999879         0.000061  
8        0.009235          1.000000         0.000000  
5        0.012127          0.998200         0.000423

In [38]:
grid_search.best_params_

{'C': 1, 'loss': 'squared_hinge'}

In [40]:
optimised_svm = grid_search.best_estimator_
print(optimised_svm)

LinearSVC(C=1)


## Matriz de confusión 
Una forma de ver fácilmente el resultado de un clasificador es utilizando una matriz de confusión. A continuación
se presenta una función para visualizar una matriz de confusión utilizando `matplotlib`.

In [41]:
import numpy as np
import itertools
import matplotlib.pylab as plt
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()



**3) Mostrá** los resultados de la predicción del clasificador `LinearSVC` sobre el conjunto de test con una matriz de confusión

In [42]:
# Mostrá la matriz de confusión en esta celda

#todo texto del metricas de la entrega 4

for title, normalize in titles_options:
    disp = plot_confusion_matrix(grid_search, tfidf_matrix, y_train,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()

NameError: name 'titles_options' is not defined

In [20]:
# Mostrá el roc_auc score y el classification_report en esta celda

#roc_auc_score(y_true, y_score)

### Random Forests
El siguiente modelo que vamos a entrenar es utilizando la técnica de Random Forest. 

__5) Usando grid search__ con cross validation (5 folds), encontrar el mejor modelo según el AUC dado el siguiente espacio de búsqueda:

* n_estimators = [100, 500, 1000]
* max_depth: [3, 7, None]
* min_samples_split: [2, 3, 10]
* bootstrap: [True, False]
* criterion: ["gini", "entropy"]

(puede tardar un poco la ejecución dado el tamaño del dataset)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification


rfc=RandomForestClassifier(random_state=42)

param_grid = { 
    'n_estimators': [100, 500, 1000],
    'max_features': [3, 7, None],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}


grid_search = GridSearchCV(rfc, param_grid, cv=5,
                           scoring='roc_auc', 
                           return_train_score=True)
grid_search.fit(tfidf_matrix, y_train)


**6) Repetir** los mismos pasos utilizados con el anterior modelo pero esta vez para `Random Forest`. Es decir, mostrar la matriz de confusión, scores, etc

In [ ]:
grid_search.get_params().keys()
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_params_
optimised_svm = grid_search.best_estimator_

#falta la matriz de confusión

### Multilayer Perceptron

**6) Repetir** los mismos pasos utilizados con el anterior modelo pero esta vez para `Multilayer Perceptron`. Es decir, mostrar la matriz de confusión, scores, etc

El `param_grid` debe recorrer estos valores: `{'hidden_layer_sizes': [(10,),(5,5), (2,2)]}`

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=42)

param_grid = {'hidden_layer_sizes': [(10,),(5,5), (2,2)]}

grid_search = GridSearchCV(clf, param_grid, cv=5,
                           scoring='roc_auc', 
                           return_train_score=True)
grid_search.fit(tfidf_matrix, y_train)


In [ ]:
grid_search.get_params().keys()
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_params_
optimised_svm = grid_search.best_estimator_

#falta la matriz de confusión

### AdaBoost

**7) Repetir** los mismos pasos utilizados con el anterior modelo pero esta vez para `AdaBoost`. Es decir, mostrar la matriz de confusión, scores, etc

El `param_grid` debe recorrer estos valores: `{'n_estimators': [50, 100, 500]}`


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(random_state=42)

param_grid = {'n_estimators': [50, 100, 500]}

grid_search = GridSearchCV(clf, param_grid, cv=5,
                           scoring='roc_auc', 
                           return_train_score=True)
grid_search.fit(tfidf_matrix, y_train)


In [ ]:
grid_search.get_params().keys()
sorted(grid_search.grid_scores_, key=lambda x: x.mean_validation_score)
grid_search.best_params_
optimised_svm = grid_search.best_estimator_

#falta la matriz de confusión

__8) Armar un ensamble entre los cuatro modelos usando votación y medir el accuracy de cada uno de los modelos.__


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

svc_clf = SVC(kernel='linear', probability=True) #  para soft
sgd_clf = SGDClassifier(loss='log') #  para soft
voting_clf = VotingClassifier(
estimators=[('svc', svc_clf), ('sgd', sgd_clf)],voting='soft')


for clf in (svc_clf, sgd_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

__9) Seleccioná el modelo__ que te haya dado el mejor score y utilizalo como tu modelo final. Reentrenalo con todos tus datos de entrenamiento.


In [ ]:
# Guardamos el modelo
from sklearn.externals import joblib
joblib.dump(eclf, 'sentiment.pkl') 

In [ ]:
y_pred

## Clasificación de reviews de Rocky 4.

A partir de [criticas de la película Rocky 4](https://www.rottentomatoes.com/m/rocky_iv/reviews/?type=user) veamos como las clasifica nuestro mejor modelo

In [ ]:
reviews_new = ["Stallone creates credible villains worthy of his heroic character.",
               "Another brilliant Rocky film, probably my favourite one out of the lot",
               "Yeah, this movie sucks.",
               "My favourite rocky film! So good so much heart. Slightly better than 2",
               "What has this got to do with boxing. Also everyone looked like dolls. Also if you are a real true boxing fan (not casuals), you would understand that this stupidity is no description of boxing!!",
               "The new film's narrative is stripped down to essentials, which gives it an emblematic quality.",
               "Absurdly ridiculous, this just isn't a good movie at all", 
               "Very basic and predictable but still an okay movie. No special music to save this one.", 
              "Rocky 4 is an extremely ambitious movie that is definitely worth watching.",
              'Highly beautiful',
               "If it wasn't for the robots (WTF????), and the painfully overwritten lines of an absurdly dogmatic persuasion, then this would otherwise be nothing more than an interminable series of mildly rousing montages. There are some unintentionally funny bits though, and Dolph's Ivan showcases the best and worst of all Rocky's opponents.",
              "While all aspects of realism is thrown out the window, ROCKY IV is an adrenaline rush of action and excitment, with an incredible soundtrack and arguably the best movie fight in history between Balboa and Drago",
              "Just like the songs, exercise routines and repetitive clips, it seems redundant to add another installment in this already falling franchise when you clearly lack material. Rocky IV is petty, childish and seems overlong despite of its 91 minutes of run time for it merely has an idea of a TV drama episode which is stretched to a point of exhaustion. Its painful to watch Sylvester Stallone go through this enormous amount of training and hardly make a point out there. He fails on all the levels here; writer, director and actor, to deliver any loose end of the thread for the audience to hang on to. Rocky IV is predictable, loosely written and choreographed and executed unsupervised."]
reviews_new_counts = tfidf_vect.transform(reviews_new)
reviews_new_tfidf = tfidf_vect.transform(reviews_new)

In [ ]:
pred = eclf.predict(reviews_new_tfidf)

In [ ]:
for review, category in zip(reviews_new, pred):
    print('%r => %s \n' % (review, movie_reviews.target_names[category]))